In [ ]:
import pandas as pd
df = pd.read_csv('/content/ctgan_result_oversampling_3.csv')

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
X = df.loc[:, df.columns !=  'DEATH_EVENT']
Y = df['DEATH_EVENT']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2,random_state=0)

In [ ]:
print(len(X_test))

82


In [ ]:
all_model_score_cum_sum = [0] * len(X_test)
# This function calculates majority voting from the classifiers
def compute_score(model, X_train,y_train, X_test):
  # for col in X_test.columns:
  #   if col not in X_train.columns:
  #     test.drop(col, axis=1, inplace=True)

  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  for i in range(0,len(all_model_score_cum_sum),1):
    all_model_score_cum_sum[i] += y_pred[i]

In [ ]:
model_xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
model_mlp = MLPClassifier(random_state=22, max_iter=300)
model_rf = RandomForestClassifier(max_depth=8, random_state=22)
model_dt = DecisionTreeClassifier(random_state=22, max_depth=3)
model_lr = LogisticRegression(random_state= 22)
model_svm = make_pipeline(StandardScaler(), SVC(gamma='auto'))
model_knn = KNeighborsClassifier(n_neighbors=47)
model_ada = AdaBoostClassifier(n_estimators=100, random_state=0)
model_gb = GradientBoostingClassifier(n_estimators=100, random_state=0)
model_et = ExtraTreesClassifier(n_estimators=100, random_state=0)

In [ ]:
model_array = [model_xgb, model_ada, model_rf, model_gb, model_et]
for model in model_array:
  compute_score(model, X_train, Y_train, X_test)

In [ ]:
all_model_score_cum_sum = [0 if x <=2 else 1 for x in all_model_score_cum_sum]
print(sum(all_model_score_cum_sum))
print(classification_report(Y_test,all_model_score_cum_sum, digits=4))

48
              precision    recall  f1-score   support

           0     0.8824    0.8571    0.8696        35
           1     0.8958    0.9149    0.9053        47

    accuracy                         0.8902        82
   macro avg     0.8891    0.8860    0.8874        82
weighted avg     0.8901    0.8902    0.8900        82



## Majority Voting with certain features

In [ ]:
# X = df.loc[:, df.columns !=  'DEATH_EVENT']
# X = X[['anaemia', 'diabetes', 'ejection_fraction', 'high_blood_pressure', 'serum_creatinine', 'serum_sodium', 'sex', 'time', 'creatinine_phosphokinase']]
# Y = df['DEATH_EVENT']
X = df.loc[:, df.columns !=  'DEATH_EVENT']
X = X[['ejection_fraction','serum_creatinine','time']]
Y = df['DEATH_EVENT']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2,random_state=0)

In [ ]:

print(len(X_test))

82


In [ ]:
all_model_score_cum_sum = [0] * len(X_test)
# This function calculates majority voting from the classifiers
def compute_score(model, X_train,y_train, X_test):
  # for col in X_test.columns:
  #   if col not in X_train.columns:
  #     test.drop(col, axis=1, inplace=True)

  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  for i in range(0,len(all_model_score_cum_sum),1):
    all_model_score_cum_sum[i] += y_pred[i]

In [ ]:
# model_xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
# model_mlp = MLPClassifier(random_state=22, max_iter=300)
# model_rf = RandomForestClassifier(max_depth=10, random_state=22)
# model_dt = DecisionTreeClassifier(random_state=22, max_depth=3)
# model_lr = LogisticRegression(random_state= 22)
# model_svm = make_pipeline(StandardScaler(), SVC(gamma='auto'))
# model_knn = KNeighborsClassifier(n_neighbors=31)

In [ ]:
all_model_score_cum_sum = [0] * len(X_test)
model_array = [model_xgb, model_ada, model_rf, model_gb, model_et,model_dt,model_lr]
for model in model_array:
  compute_score(model, X_train, Y_train, X_test)

In [ ]:

model_array = [model_xgb, model_gb, model_et, ]
for model in model_array:
  compute_score(model, X_train, Y_train, X_test)

In [ ]:
all_model_score_cum_sum = [0 if x <=1 else 1 for x in all_model_score_cum_sum]
print(sum(all_model_score_cum_sum))
print(classification_report(Y_test,all_model_score_cum_sum,digits=4))

41
              precision    recall  f1-score   support

           0     0.7805    0.9143    0.8421        35
           1     0.9268    0.8085    0.8636        47

    accuracy                         0.8537        82
   macro avg     0.8537    0.8614    0.8529        82
weighted avg     0.8644    0.8537    0.8544        82

